In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
class FullyConnectedNN(nn.Module):
    def __init__(self, input_size=16384, hidden_size=512, output_size=1):
        super(FullyConnectedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        return out

In [3]:
# Подготовка данных (пример)
X = pd.read_csv(r'C:\PycharmProjects\ETS_Autopilot\dataset_for_wheel_nn\X_train_128_128.csv')
y = pd.read_csv(r'C:\PycharmProjects\ETS_Autopilot\dataset_for_wheel_nn\y_train_128_128.csv')

In [4]:
X.iloc[0]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
16379    0.0
16380    0.0
16381    0.0
16382    0.0
16383    0.0
Name: 0, Length: 16384, dtype: float64

In [5]:
y.head(5)

,value
0,-0.013916
1,-0.008545
2,-0.007690
3,-0.003662
4,-0.003662


In [6]:
Xi = torch.tensor(X.values, dtype=torch.float32).cuda() 
yi = torch.tensor(y.values, dtype=torch.float32).cuda() 

In [7]:
X_tensor = Xi
y_tensor = yi

In [8]:
len(Xi[0])

16384

In [9]:
yi[1]

tensor([-0.0085], device='cuda:0')

In [10]:
model = FullyConnectedNN().cuda()

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()  # Функция потерь для регрессии
optimizer = optim.Adam(model.parameters(), lr=0.000001)  # Оптимизатор Adam с learning rate 0.001

# Обучение модели
num_epochs = 4
for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(len(X_tensor)):
        inputs = X_tensor[i]
        labels = torch.tensor([y_tensor[i].item()], dtype=torch.float32).cuda()

        # Обнуление градиентов параметров
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(inputs)

        # Рассчет функции потерь
        loss = criterion(outputs, labels)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(X_tensor)}")

print("Обучение завершено")

Epoch [1/4], Loss: 0.00997986793780339
Epoch [2/4], Loss: 0.009956634291055656
Epoch [3/4], Loss: 0.012837211974677725
Epoch [4/4], Loss: 0.009951265561525804
Обучение завершено


In [11]:
model.forward(X_tensor[400])

tensor([0.0858], device='cuda:0', grad_fn=<TanhBackward0>)

In [12]:
model.forward(X_tensor[3])

tensor([0.0857], device='cuda:0', grad_fn=<TanhBackward0>)

In [13]:
torch.save(model.state_dict(), 'C:\PycharmProjects\ETS_Autopilot\dataset_for_wheel_nn\weight_128_128.pth')